In [241]:
import pandas as pd 
import os 
from dotenv import load_dotenv
import requests 
import json 
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [242]:
load_dotenv()

True

In [243]:
api_key = os.getenv('BLOCKSPAN_API_KEY')

In [244]:
url = "https://api.blockspan.com/v1/exchanges/collectionsranking?chain=eth-main&exchange=opensea&ranking=seven_day_volume&page_size=100"
headers = {
    "accept": "application/json",
    "X-API-KEY": api_key
}

In [245]:
response = requests.get(url, headers=headers).json()
#print(json.dumps(response, indent=4, sort_keys=True))

In [246]:
nft_df = pd.DataFrame(response['results'])
nft_df.head()

,key,name,description,update_at,total_volume,one_day_volume,seven_day_volume,thirty_day_volume,total_sales,one_day_sales,seven_day_sales,thirty_day_sales,total_average_price,one_day_average_price,seven_day_average_price,thirty_day_average_price,contracts
0,valhalla,Valhalla,Valhalla is a crypto native brand for gamers. ...,2022-12-15T23:10:01.865Z,10239.45533778399,346.0350129899999,4076.5065266468223,10239.455337783958,11462,204,3298,11462,0.8933393245318434,1.6962500636764701,1.2360541317910316,0.8933393245318407,[{'contract_address': '0x231d3559aa848bf10366f...
1,art-blocks,Art Blocks,Art Blocks is dedicated to bringing compelling...,2022-12-15T19:49:23.602Z,463304.69361787103,281.12792500000006,2136.789362268989,9127.35848823819,207755,200,1662,6494,2.230053156929417,1.4056396250000003,1.2856735031702702,1.4055063887031396,[{'contract_address': '0xa7d8d9ef8d8ce8992df33...
2,cryptopunks,CryptoPunks,"CryptoPunks launched as a fixed set of 10,000 ...",2022-12-15T21:42:01.591Z,1075232.3970093378,136.9,2008.7700000000002,13737.930000000004,22053,3,31,203,48.756740443900505,45.63333333333333,64.79903225806453,67.67453201970446,[{'contract_address': '0xb47e3cd837ddf8e4c57f0...
3,mutant-ape-yacht-club,Mutant Ape Yacht Club,The MUTANT APE YACHT CLUB is a collection of u...,2022-12-15T22:46:46.630Z,472283.55948862835,195.95708000000002,1971.5778538333332,8181.297815533331,36132,13,127,577,13.07106054158719,15.07362153846154,15.524235069553804,14.179025676834197,[{'contract_address': '0x60e4d786628fea6478f78...
4,boredapeyachtclub,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,2022-12-15T23:14:27.848Z,697241.6911093168,352.69455104,1815.7271510399999,10882.586567539995,27515,5,25,139,25.34042126510328,70.538910208,72.62908604159999,78.29198969453233,[{'contract_address': '0xbc4ca0eda7647a8ab7c20...


In [247]:
nft_df['change'] = nft_df['one_day_average_price'].astype(float) - nft_df['seven_day_average_price'].astype(float)
nft_df.head()
#If positive, then stock increased
#If negative, stock fell

,key,name,description,update_at,total_volume,one_day_volume,seven_day_volume,thirty_day_volume,total_sales,one_day_sales,seven_day_sales,thirty_day_sales,total_average_price,one_day_average_price,seven_day_average_price,thirty_day_average_price,contracts,change
0,valhalla,Valhalla,Valhalla is a crypto native brand for gamers. ...,2022-12-15T23:10:01.865Z,10239.45533778399,346.0350129899999,4076.5065266468223,10239.455337783958,11462,204,3298,11462,0.8933393245318434,1.6962500636764701,1.2360541317910316,0.8933393245318407,[{'contract_address': '0x231d3559aa848bf10366f...,0.460196
1,art-blocks,Art Blocks,Art Blocks is dedicated to bringing compelling...,2022-12-15T19:49:23.602Z,463304.69361787103,281.12792500000006,2136.789362268989,9127.35848823819,207755,200,1662,6494,2.230053156929417,1.4056396250000003,1.2856735031702702,1.4055063887031396,[{'contract_address': '0xa7d8d9ef8d8ce8992df33...,0.119966
2,cryptopunks,CryptoPunks,"CryptoPunks launched as a fixed set of 10,000 ...",2022-12-15T21:42:01.591Z,1075232.3970093378,136.9,2008.7700000000002,13737.930000000004,22053,3,31,203,48.756740443900505,45.63333333333333,64.79903225806453,67.67453201970446,[{'contract_address': '0xb47e3cd837ddf8e4c57f0...,-19.165699
3,mutant-ape-yacht-club,Mutant Ape Yacht Club,The MUTANT APE YACHT CLUB is a collection of u...,2022-12-15T22:46:46.630Z,472283.55948862835,195.95708000000002,1971.5778538333332,8181.297815533331,36132,13,127,577,13.07106054158719,15.07362153846154,15.524235069553804,14.179025676834197,[{'contract_address': '0x60e4d786628fea6478f78...,-0.450614
4,boredapeyachtclub,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,2022-12-15T23:14:27.848Z,697241.6911093168,352.69455104,1815.7271510399999,10882.586567539995,27515,5,25,139,25.34042126510328,70.538910208,72.62908604159999,78.29198969453233,[{'contract_address': '0xbc4ca0eda7647a8ab7c20...,-2.090176


In [248]:
nft_df = nft_df

In [249]:
nft_df['target'] = nft_df['change'].apply(lambda x: 0 if x <= 0 else 1)


In [250]:
new_nft_df = nft_df.set_index('key')
new_nft_df = nft_df.drop(columns=['update_at','name','description','contracts','key'])

In [251]:
# The y variable should focus on the target column
y = new_nft_df['target'].astype(float)

# The X variable should include all features except the target
x = new_nft_df.drop(columns=['target']).astype(float)



In [252]:
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [253]:
#Scaling data sets
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
#trans X train and test sets using scaler
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [254]:
# Declare a logistic regression model.
# Apply a random_state of 9 to the model
logistic_regression_model = LogisticRegression(random_state=9)
logistic_regression_model

LogisticRegression(random_state=9)

In [255]:
# Fit and save the logistic regression model using the training data
lr_model = logistic_regression_model.fit(x_train, y_train)

/Users/alexandrapaiz/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [256]:
#Generate training predictions
training_predictions = lr_model.predict(X_train)

#Generate testing predictions
testing_predictions = logistic_regression_model.predict(X_test)

/Users/alexandrapaiz/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
/Users/alexandrapaiz/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"


In [257]:
# Create and save the confustion matrix for the training data
training_matrix = confusion_matrix(y_train, training_predictions)

# Print the confusion matrix for the training data
print(training_matrix)

[[12 36]
 [10 17]]


In [258]:
# Create and save the confustion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[ 3 14]
 [ 4  4]]


In [259]:
# Create and save the training classifiction report
training_report = classification_report(y_train, training_predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.55      0.25      0.34        48
         1.0       0.32      0.63      0.42        27

    accuracy                           0.39        75
   macro avg       0.43      0.44      0.38        75
weighted avg       0.46      0.39      0.37        75



In [260]:
# Create and save the testing classifiction report
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.43      0.18      0.25        17
         1.0       0.22      0.50      0.31         8

    accuracy                           0.28        25
   macro avg       0.33      0.34      0.28        25
weighted avg       0.36      0.28      0.27        25

